In [1]:
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LEARNING_RATE = 1e-5

In [2]:
### training set 있는 Path ###
BASE_PATH = '../data/train/'

In [3]:
import torchvision.transforms as transforms
### Transforms ###
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.CenterCrop(300)
])

## Load dataset

In [4]:
from Dataset import TrainValidDataset

In [5]:
import pandas as pd
df = pd.read_csv(BASE_PATH + 'train.csv')

In [6]:
# training set (validation set 따로 안 나눈 경우)
# train_dataset = AllClassDataset(
#     base_path = BASE_PATH, 
#     data = df, 
#     transform = transform
# )

In [7]:
# train / validation data
from sklearn.model_selection import train_test_split
def mapAgeGender(age, gender):
    answer = 0
    if age < 30:
        answer += 0
    elif age >= 60:
        answer += 2
    else:
        answer += 1
    return answer if gender == 'male' else answer + 3

y_data = df.apply(lambda x: mapAgeGender(x['age'], x['gender']), axis=1)
x_train, x_val, y_train, y_val = train_test_split(df.index, y_data, test_size=0.2, random_state=42, stratify=y_data)

In [8]:
age_dataset = TrainValidDataset(
    base_path = BASE_PATH, 
    data = df.loc[x_train], 
    transform = transform,
    label="age"
)

100%|██████████| 2160/2160 [00:17<00:00, 125.76it/s]


In [9]:
age_val_dataset = TrainValidDataset(
    base_path = BASE_PATH, 
    data = df.loc[x_val], 
    transform = transform,
    label="age"
)

100%|██████████| 540/540 [00:03<00:00, 141.29it/s]


### Dataset 확인용 Code
```python
import matplotlib.pyplot as plt
image, label = val_dataset[0]
plt.title(f'Class {label}')
plt.imshow(image.permute(1,2,0))
plt.show()
```

## DataLoader

In [10]:
age_trainloader = torch.utils.data.DataLoader(
    age_dataset,
    batch_size=64,
    num_workers=1
)

In [11]:
age_valloader = torch.utils.data.DataLoader(
    age_val_dataset,
    batch_size=64,
    num_workers=1
)

### Dataloader 확인용 Code
```python
import matplotlib.pyplot as plt
images, labels = next(iter(trainloader))
plt.title(f'Class {labels[0]}')
plt.imshow(images[0].permute(1,2,0))
plt.show()
```

## Model

In [12]:
from Model import ModifiedModel

In [13]:
age_model = ModifiedModel(num_classes = 3).to(DEVICE)

In [14]:
age_optimizer = torch.optim.Adam(age_model.parameters(), lr=LEARNING_RATE)

## Train

In [15]:
from train import train

In [16]:
# Tensorboard
from torch.utils.tensorboard import SummaryWriter
train_writer = SummaryWriter('runs/train_resnet18_age')
val_writer = SummaryWriter('runs/val_resnet18_age')

In [17]:
### Parameter 변경 ###
train(
    model=age_model, 
    optimizer=age_optimizer,
    train_loader=age_trainloader, 
    val_loader=age_valloader,
    device=DEVICE, 
    epochs=30, 
    save=True, 
    saved_folder="saved/age",
    train_writer=train_writer,
    val_writer=val_writer
)

  0%|          | 0/237 [00:00<?, ?it/s]

Epoch 001: Loss: 0.6533 / Acc: 0.734 / F1: 0.59        | Val Loss: 0.3132 / Val Acc: 0.882 / Val F1: 0.77


  0%|          | 0/237 [00:00<?, ?it/s]

Epoch 002: Loss: 0.3120 / Acc: 0.886 / F1: 0.78        | Val Loss: 0.2643 / Val Acc: 0.899 / Val F1: 0.80


  0%|          | 0/237 [00:00<?, ?it/s]

...

Epoch 028: Loss: 0.0011 / Acc: 1.000 / F1: 1.00        | Val Loss: 0.4717 / Val Acc: 0.903 / Val F1: 0.79


  0%|          | 0/237 [00:00<?, ?it/s]

Epoch 029: Loss: 0.0011 / Acc: 1.000 / F1: 1.00        | Val Loss: 0.4486 / Val Acc: 0.910 / Val F1: 0.80


100%|██████████| 237/237 [01:05<00:00,  3.62it/s]


Epoch 030: Loss: 0.0096 / Acc: 0.998 / F1: 1.00        | Val Loss: 0.4978 / Val Acc: 0.894 / Val F1: 0.77


In [ ]:
train_writer.close()
val_writer.close()

In [ ]:
# rm -rf /opt/ml/.cache/torch/hub/checkpoints/